In [2]:
import json
import requests
from bs4 import BeautifulSoup
import re
from pprint import pprint
import pandas as pd

In [29]:
df = pd.read_csv('Presidential_Elections_Data - 2020.csv')

In [30]:
df

,STATE,TOTAL VOTES,Rep_Votes,Rep_Percent,Rep_EV,Dem_Votes,Dem_Percent,Dem_EV,Others_Votes,Others_Percent,YEAR
0,Alabama,"2,323,282","1,441,170",62.03%,9,"849,624",36.57%,NaN,"32,488",1.40%,2020
1,Alaska,"359,530","189,951",52.83%,3,"153,778",42.77%,NaN,"15,801",4.39%,2020
2,Arizona,"3,387,326","1,661,686",49.06%,NaN,"1,672,143",49.36%,11,"53,497",1.58%,2020
3,Arkansas,"1,219,069","760,647",62.40%,6,"423,932",34.78%,NaN,"34,490",2.83%,2020
4,California,"17,500,881","6,006,429",34.32%,NaN,"11,110,250",63.48%,55,"384,202",2.20%,2020
...,...,...,...,...,...,...,...,...,...,...,...
309,Virginia,"2,739,447","1,437,490",52.5,13,"1,217,290",44.4,NaN,"59,398",2.2,2000
310,Washington,"2,487,433","1,108,864",44.6,NaN,"1,247,652",50.2,11,"103,002",4.1,2000
311,West Virginia,"648,124","336,475",51.9,5,"295,497",45.6,NaN,"10,680",1.6,2000
312,Wisconsin,"2,598,607","1,237,279",47.6,NaN,"1,242,987",47.8,11,"94,070",3.6,2000


In [31]:
df = df.replace(to_replace=r'\*$', value='', regex=True)

df['Rep_Percent'] = pd.to_numeric(df['Rep_Percent'].str.rstrip('%'))
df['Dem_Percent'] = pd.to_numeric(df['Dem_Percent'].str.rstrip('%'))

df['Victory_Margin'] = df.apply(lambda row: abs(row['Rep_Percent'] - row['Dem_Percent']), axis=1)
df['Victory_Margin'] = df['Victory_Margin'].round(2)

In [32]:
df

,STATE,TOTAL VOTES,Rep_Votes,Rep_Percent,Rep_EV,Dem_Votes,Dem_Percent,Dem_EV,Others_Votes,Others_Percent,YEAR,Victory_Margin
0,Alabama,"2,323,282","1,441,170",62.03,9,"849,624",36.57,NaN,"32,488",1.40%,2020,25.46
1,Alaska,"359,530","189,951",52.83,3,"153,778",42.77,NaN,"15,801",4.39%,2020,10.06
2,Arizona,"3,387,326","1,661,686",49.06,NaN,"1,672,143",49.36,11,"53,497",1.58%,2020,0.30
3,Arkansas,"1,219,069","760,647",62.40,6,"423,932",34.78,NaN,"34,490",2.83%,2020,27.62
4,California,"17,500,881","6,006,429",34.32,NaN,"11,110,250",63.48,55,"384,202",2.20%,2020,29.16
...,...,...,...,...,...,...,...,...,...,...,...,...
309,Virginia,"2,739,447","1,437,490",52.50,13,"1,217,290",44.40,NaN,"59,398",2.2,2000,8.10
310,Washington,"2,487,433","1,108,864",44.60,NaN,"1,247,652",50.20,11,"103,002",4.1,2000,5.60
311,West Virginia,"648,124","336,475",51.90,5,"295,497",45.60,NaN,"10,680",1.6,2000,6.30
312,Wisconsin,"2,598,607","1,237,279",47.60,NaN,"1,242,987",47.80,11,"94,070",3.6,2000,0.20


In [33]:
df.to_csv('Presidential_Elections_With_Victory_Margins.csv', index=False)

In [34]:
def calculate_weighted_score(row):
    if row['YEAR'] == 2020:
        return row['Victory_Margin'] * 50
    elif row['YEAR'] == 2016:
        return row['Victory_Margin'] * 25
    elif row['YEAR'] == 2012:
        return row['Victory_Margin'] * 12.5
    elif row['YEAR'] == 2008:
        return row['Victory_Margin'] * 6.25
    elif row['YEAR'] == 2004:
        return row['Victory_Margin'] * 3.125
    elif row['YEAR'] == 2000:
        return row['Victory_Margin'] * 1.5625

# Apply the function to add the 'Weighted_Score' column
df['Weighted_Score'] = df.apply(calculate_weighted_score, axis=1).round(2)

df.head(20)

,STATE,TOTAL VOTES,Rep_Votes,Rep_Percent,Rep_EV,Dem_Votes,Dem_Percent,Dem_EV,Others_Votes,Others_Percent,YEAR,Victory_Margin,Weighted_Score
0,Alabama,"2,323,282","1,441,170",62.03,9,"849,624",36.57,NaN,"32,488",1.40%,2020,25.46,1273.0
1,Alaska,"359,530","189,951",52.83,3,"153,778",42.77,NaN,"15,801",4.39%,2020,10.06,503.0
2,Arizona,"3,387,326","1,661,686",49.06,NaN,"1,672,143",49.36,11,"53,497",1.58%,2020,0.30,15.0
3,Arkansas,"1,219,069","760,647",62.40,6,"423,932",34.78,NaN,"34,490",2.83%,2020,27.62,1381.0
4,California,"17,500,881","6,006,429",34.32,NaN,"11,110,250",63.48,55,"384,202",2.20%,2020,29.16,1458.0
5,Colorado,"3,256,952","1,364,607",41.90,NaN,"1,804,352",55.40,9,"87,993",2.70%,2020,13.50,675.0
6,Connecticut,"1,824,280","715,291",39.21,NaN,"1,080,680",59.24,7,"28,309",1.55%,2020,20.03,1001.5
7,Delaware,"504,010","200,603",39.80,NaN,"296,268",58.78,3,"7,139",1.42%,2020,18.98,949.0
8,District of Columbia,"344,356","18,586",5.40,NaN,"317,323",92.15,3,"8,447",2.45%,2020,86.75,4337.5
9,Florida,"11,067,456","5,668,731",51.22,29,"5,297,045",47.86,NaN,"101,680",0.92%,2020,3.36,168.0


In [35]:
df_sorted = df.sort_values(by='Weighted_Score', ascending=True)

df_sorted

,STATE,TOTAL VOTES,Rep_Votes,Rep_Percent,Rep_EV,Dem_Votes,Dem_Percent,Dem_EV,Others_Votes,Others_Percent,YEAR,Victory_Margin,Weighted_Score
272,Florida,"5,963,110","2,912,790",48.80,25,"2,912,253",48.80,NaN,"97,488",1.6,2000,0.00,0.00
294,New Mexico,"598,605","286,417",47.80,NaN,"286,783",47.90,5,"21,251",3.6,2000,0.10,0.16
312,Wisconsin,"2,598,607","1,237,279",47.60,NaN,"1,242,987",47.80,11,"94,070",3.6,2000,0.20,0.31
278,Iowa,"1,315,563","634,373",48.20,NaN,"638,517",48.50,7,"29,374",2.2,2000,0.30,0.47
186,Missouri,"2,925,205","1,445,814",49.40,11,"1,441,911",49.30,NaN,NaN,NaN,2008,0.10,0.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,West Virginia,"794,652","545,382",68.63,5,"235,984",29.70,NaN,"13,286",1.67%,2020,38.93,1946.50
55,Wyoming,"276,765","193,559",69.94,3,"73,491",26.55,NaN,"9,715",3.51%,2020,43.39,2169.50
64,Dist. of Col.,"311,268","12,723",4.10,NaN,"282,830",90.90,3,"4,906",1.60%,2016,86.80,2170.00
32,CD-3,"294,831","222,179",75.36,1,"65,854",22.34,NaN,"6,798",2.31%,2020,53.02,2651.00


In [36]:
df_sorted.to_csv('Presidential_Elections_With_Weighted_Scores.csv', index=False)